In [1]:
from utils.all import *
log.basicConfig(level=log.INFO)
data = load_data('../data/inspection_rating_annotated', preprocess=True, adjudicated=True)

DEBUG:matplotlib data path: C:\Users\heine\AppData\Roaming\Python\Python39\site-packages\matplotlib\mpl-data
DEBUG:CONFIGDIR=C:\Users\heine\.matplotlib
DEBUG:interactive is False
DEBUG:platform is win32
DEBUG:CACHEDIR=C:\Users\heine\.matplotlib
DEBUG:Using fontManager instance from C:\Users\heine\.matplotlib\fontlist-v330.json
INFO:Loading files: ['../data/inspection_rating_annotated/batch_1_ayush.json', '../data/inspection_rating_annotated/batch_1_rachel.json', '../data/inspection_rating_annotated/batch_1_vinayak.json', '../data/inspection_rating_annotated/batch_1_vishnesh.json', '../data/inspection_rating_annotated/batch_2_ayush.json', '../data/inspection_rating_annotated/batch_2_rachel.json', '../data/inspection_rating_annotated/batch_2_vinayak.json', '../data/inspection_rating_annotated/batch_2_vishnesh.json', '../data/inspection_rating_annotated/batch_3_ayush.json', '../data/inspection_rating_annotated/batch_3_rachel.json', '../data/inspection_rating_annotated/batch_3_vishnesh.jso

In [40]:
def count_structure_change(data):
    total_count = {sys: None for sys in set([s['system'] for s in data])}
    for system in total_count.keys():
        s_count = {s_type: {q_type: 0 for q_type in Quality} for s_type in Structure}

        # Get all structure annotations
        structure_anns = [ann for ann in [i for j in [sent['processed_annotations'] for sent in data if sent['system'] == system] for i in j] if ann['edit_type'] == 'structure']
        structure_anns = [ann for ann in structure_anns if ann['subtype'] != '' and ann['subtype'] is not None]

        for ann in structure_anns:
            s_count[ann['subtype']][ann['type']] += 1
        total_count[system] = s_count
    return total_count

In [41]:
structure_change_count = count_structure_change(data)

In [42]:
structure_change_count

{'new-wiki-1/GPT-3-few-shot': {<Structure.VOICE: 'Voice'>: {<Quality.QUALITY: 'No Error'>: 30,
   <Quality.TRIVIAL: 'Trivial'>: 2,
   <Quality.ERROR: 'Error'>: 4},
  <Structure.POS: 'Part of Speech'>: {<Quality.QUALITY: 'No Error'>: 50,
   <Quality.TRIVIAL: 'Trivial'>: 7,
   <Quality.ERROR: 'Error'>: 6},
  <Structure.TENSE: 'Tense'>: {<Quality.QUALITY: 'No Error'>: 131,
   <Quality.TRIVIAL: 'Trivial'>: 8,
   <Quality.ERROR: 'Error'>: 3},
  <Structure.GRAMMAR_NUMBER: 'Grammatical Number'>: {<Quality.QUALITY: 'No Error'>: 7,
   <Quality.TRIVIAL: 'Trivial'>: 6,
   <Quality.ERROR: 'Error'>: 1},
  <Structure.CLAUSAL: 'Clausal Structure'>: {<Quality.QUALITY: 'No Error'>: 123,
   <Quality.TRIVIAL: 'Trivial'>: 38,
   <Quality.ERROR: 'Error'>: 14},
  <Structure.TRANSITION: 'Transition'>: {<Quality.QUALITY: 'No Error'>: 12,
   <Quality.TRIVIAL: 'Trivial'>: 5,
   <Quality.ERROR: 'Error'>: 0}},
 'new-wiki-1/T5-3B': {<Structure.VOICE: 'Voice'>: {<Quality.QUALITY: 'No Error'>: 1,
   <Quality.TRIVIAL

In [ ]:
fam = edit_ratings_by_family(data, size_weighted=size_weighted)
sys = list(list(fam.values())[0].keys())

if include_all:
    plt.rcParams["figure.figsize"] = [12.5, 4]
else:
    fam.pop('all', None)
    plt.rcParams["figure.figsize"] = [11, 4]

family_order = [
    'elaboration',
    'generalization',
    'paraphrase',
    'reorder',
    'split',
    'structure',
    'all'
]

custom_family_mapping = {
    'elaboration': 'More Information',
    'generalization': 'Less Information',
    'paraphrase': 'Same Information',
    'reorder': 'Reorder',
    'split': 'Sentence Split',
    'structure': 'Structure',
    'all': 'All',
}

fig, ax = plt.subplots(len(sys), len(fam.keys()))
for i, family in enumerate(family_order):
    ratings = fam[family]

    for j, system in enumerate([s for s in all_system_labels if s in ratings.keys()]):
        left = 0
        curr_plots = []
        for k, rating in enumerate(ratings[system]):
            if k < 3:
                color = '#5e5e5e'
            elif k > 3:
                color = color_mapping[family]
            else:
                color = '#b8b398'

            if i == 0:
                label = system_name_mapping[system]
                if 'GPT' in label:
                    label = label.replace(' GPT', '\nGPT')
                ax[j, i].tick_params(left=False, labelsize=12)
            else:
                label = ' '
                ax[j, i].axis('off')
            
            dem = 3
            scalar = [0.6, 1, 1.3, 1, 0.8, 1, 1.2]
            color = colorscale(color, scalar[k]) # abs(3-k)*(1/dem) + (1/dem)

            bar_plot = ax[j, i].barh(label, rating, left=left, color=color, edgecolor='#cfcfcf',) ##6e6e6e # alpha=abs(3-k)*(1/dem) + (1/dem)
            ax[j, i].tick_params(labelsize=9)
            
            ax[j, i].add_patch(mpl.patches.Rectangle((0, -0.4), 1, 0.8, fill=None, alpha=1))
            
            curr_plots += [bar_plot]
            left += rating
        if j == 0:
            # add padding
            ax[j, i].set_title(custom_family_mapping[family], pad=10)
        elif j == len(ratings.keys()) - 1:
            labels = [f'–{abs(x-3)} ' if x - 3 < 0 else f'+{x-3}' for x in range(7)]
            legend = ax[j, i].legend(
                handles=curr_plots, labels=labels, bbox_to_anchor=(0.5, -1.2), 
                loc='lower center', borderaxespad=0.,fontsize=10,ncol=7,
                facecolor='white',edgecolor='black',framealpha=1,frameon=False,
                columnspacing=0.1,handlelength=1,handleheight=1,
                # Should be 0
                handletextpad=-1, borderpad=0, title=' Error         Quality'
            )
            handles, labels = ax[j, i].get_legend_handles_labels()
            for ha in ax[j, i].legend_.legendHandles:
                ha.set_edgecolor("black")
            for text in legend.get_texts():
                text.set_position((0, -15))

        ax[j, i].set_xticks([])

        ax[j, i].spines['bottom'].set_visible(False)
        ax[j, i].spines['top'].set_visible(False)
        ax[j, i].spines['left'].set_visible(False)
        ax[j, i].spines['right'].set_visible(False)

plt.subplots_adjust(wspace=0, hspace=0, bottom=0.5)

# Arbitrary text to force padding at bottom of figure
fig.text(0.5, -0.01, ' ', ha='center', fontsize=14)

plt.tight_layout()
plt.subplots_adjust()

if size_weighted:
    out_filename = "../paper/plot/appendix/structure-scores.pdf"
else:
    out_filename = "../paper/plot/appendix/structure-scores-non-weighted.pdf"
plt.savefig(out_filename, format="pdf", bbox_inches='tight', pad_inches=0.0)

plt.show()